# US ETFs Performance Analysis with SQL and Tableau

This notebook serves as a comprehensive guide to understanding the dataset and offers practical techniques for establishing a connection between Jupyter notebook and MySQL, facilitating data exploration within a Python environment.

## Table of Content:
* [1. Data Description](#data_describe)
* [2. Connecting MySQL with Jupyter notebook](#conn_sql)
* [3. Data Exploration](#data_explore)
* [4. Tableau Visualization](#tableau)

## 1. Data Description <a class="anchor" id="data_describe"></a>

The dataset contains Exchange-Traded Funds (ETFs), which offer a cost-effective alternative to mutual funds. ETFs have experienced rapid growth in recent years due to their passive investment approach, resulting in lower fees. 

The dataset comprises financial data sourced from Yahoo Finance, encompassing all U.S. ETFs along with their historical prices. The updated version reflects financial values as of November 2021.

Below are some description on the key attributes (columns):

### General Information
- Fund_symbol: A ticker symbol, also known as a stock symbol, serves as a unique identifier for publicly traded shares on stock markets, typically comprising a combination of letters and numbers.
- Inception_date: The inception date denotes the commencement of operations for a fund, marking the initial date it became available to investors.
- Category: Fund categories provide a method for classifying mutual funds based on their investment objectives and key features, allowing investors to diversify their portfolios across a range of risk and return profiles.

### Investment_type: One of three investment types.
- Growth Fund: These funds primarily invest in stocks anticipated to appreciate in value, prioritizing capital growth over high income yields.
- Value Fund: A value fund concentrates on investing in undervalued stocks based on fundamental criteria, seeking shares priced below their intrinsic value for various reasons.
- Blend Fund: Blend funds, composed solely of stocks without fixed-income securities, combine growth and value stocks within their portfolios.                                                              

### Financial Metrics      
- Price_book_ratio: The price-to-book ratio (P/B ratio) compares a company's market value to its book value, offering insights into its valuation.
- Net_asset_value: This metric reflects the value of a mutual fund by subtracting liabilities from the market value of all its shares and dividing by the number of issued shares.
- Fund_return_ytd: Fund return year-to-date represents the profit or loss generated by a fund's investments during the current calendar year.
- Fund_alpha_3years: Alpha over a three-year period measures a fund's performance relative to risk-adjusted benchmarks, often indicating the value a portfolio manager adds or subtracts.
- Fund_beta_3years: Beta over three years gauges a fund's volatility compared to the broader market, with values exceeding one indicating greater volatility.
- Fund_sharpe_ratio_3years: This ratio assesses a fund's risk-adjusted performance over three years, comparing its returns to those of a risk-free asset after adjusting for risk.
- avg_vol_3month: The average trading volume observed during the most recent three-month period provides insights into market activity and liquidity levels.            
                    
Data sources: https://www.kaggle.com/code/arvinthsss/mutual-funds-project/notebook

## 2. Connecting MySQL with Jupyter notebook <a class="anchor" id="conn_sql"></a>

In [1]:
# Read the file
import pandas as pd
df_funds = pd.read_csv('data/us_etfs_data.csv')
df_funds.head()

,fund_symbol,quote_type,region,fund_short_name,fund_long_name,currency,fund_category,fund_family,exchange_code,exchange_name,...,fund_stdev_5years,fund_sharpe_ratio_5years,fund_treynor_ratio_5years,fund_alpha_10years,fund_beta_10years,fund_mean_annual_return_10years,fund_r_squared_10years,fund_stdev_10years,fund_sharpe_ratio_10years,fund_treynor_ratio_10years
0,AAAU,ETF,US,DWS RREEF Real Assets Fund - Cl,DWS RREEF Real Assets Fund - Class A,USD,NaN,DWS,PCX,NYSEArca,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AADR,ETF,US,AllianzGI Health Sciences Fund,Virtus AllianzGI Health Sciences Fund Class P,USD,Foreign Large Growth,Virtus,NGM,NasdaqGM,...,19.30,0.62,9.66,3.32,0.96,0.79,73.64,16.78,0.53,8.15
2,AAXJ,ETF,US,NaN,American Century One Choice Blend+ 2015 Portfo...,USD,Pacific/Asia ex-Japan Stk,American Century Investments,NGM,NasdaqGM,...,15.91,0.66,10.37,0.30,0.99,0.55,78.24,16.83,0.36,4.81
3,ABEQ,ETF,US,Thrivent Large Cap Growth Fund,Thrivent Large Cap Growth Fund Class A,USD,Large Value,Thrivent Funds,PCX,NYSEArca,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACES,ETF,US,NaN,American Century One Choice Blend+ 2015 Portfo...,USD,Miscellaneous Sector,American Century Investments,PCX,NYSEArca,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# print and convert the columns to list

df_funds.columns.to_list()

['fund_symbol',
 'quote_type',
 'region',
 'fund_short_name',
 'fund_long_name',
 'currency',
 'fund_category',
 'fund_family',
 'exchange_code',
 'exchange_name',
 'exchange_timezone',
 'avg_vol_3month',
 'avg_vol_10day',
 'total_net_assets',
 'day50_moving_average',
 'day200_moving_average',
 'week52_high_low_change',
 'week52_high_low_change_perc',
 'week52_high',
 'week52_high_change',
 'week52_high_change_perc',
 'week52_low',
 'week52_low_change',
 'week52_low_change_perc',
 'investment_strategy',
 'fund_yield',
 'inception_date',
 'annual_holdings_turnover',
 'investment_type',
 'size_type',
 'fund_annual_report_net_expense_ratio',
 'category_annual_report_net_expense_ratio',
 'asset_stocks',
 'asset_bonds',
 'fund_sector_basic_materials',
 'fund_sector_communication_services',
 'fund_sector_consumer_cyclical',
 'fund_sector_consumer_defensive',
 'fund_sector_energy',
 'fund_sector_financial_services',
 'fund_sector_healthcare',
 'fund_sector_industrials',
 'fund_sector_real_est

In [3]:
#conver file to sql

# import sqlalchemy and create a mysql engine
from sqlalchemy import create_engine

hostname = "localhost"
username = "root"
password = "Yongbaby1029!" # use own password
port = 3306
database = "us_etfs"

e = 'exception'

try:
    engine = create_engine("mysql+pymysql://"+username+":"+password+"@"+hostname+":"+str(port)+"/"+database, echo=False)
except e:
    print (e)
if engine:
    print ("connection successful")
else:
    print ("error")

# engine = create_engine('mysql+pymysql://', echo=False)

connection successful


In [4]:
# export the dataframe as a table 'us_etfs_data' to the MySQL engine
# df_funds.to_sql("us_etfs_data", con =engine, if_exists='replace', index=False)

In [6]:
## show tables in database

sql='''
SHOW TABLES
IN us_etfs
''';

df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,Tables_in_us_etfs
0,us_etfs_data
1,us_etfs_staging


In [7]:
## Test Query

sql='''
select * from us_etfs_data
limit 1
''';

df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,fund_symbol,quote_type,region,fund_short_name,fund_long_name,currency,fund_category,fund_family,exchange_code,exchange_name,...,fund_stdev_5years,fund_sharpe_ratio_5years,fund_treynor_ratio_5years,fund_alpha_10years,fund_beta_10years,fund_mean_annual_return_10years,fund_r_squared_10years,fund_stdev_10years,fund_sharpe_ratio_10years,fund_treynor_ratio_10years
0,AAAU,ETF,US,DWS RREEF Real Assets Fund - Cl,DWS RREEF Real Assets Fund - Class A,USD,None,DWS,PCX,NYSEArca,...,None,None,None,None,None,None,None,None,None,None


## 3. Data Exploration <a class="anchor" id="data_explore"></a>

The data exploration section predominantly utilizes Tableau dashboards, accessible via the provided links below. Complementing this visual analysis, this section employs SQL queries to delve deeper into specific findings showcased within the dashboards.

### 1. Overview - Investigating the Relationship between Investment Type and Fund Returns

As SQL lacks a built-in function to directly calculate the median of a column, for the sake of project simplicity, the mean is utilized as an approximation. Verification against the median reveals negligible deviation, thus establishing it as a reasonable alternative.

In [32]:
# query for fund returns by fund category

sql='''
WITH cate_size AS (
    SELECT
        investment_type,
        size_type,
        ROUND(AVG(fund_return_ytd), 2) average_return_ytd
    FROM us_etfs_staging
    WHERE fund_category IS NOT NULL
    AND size_type IS NOT NULL
    AND fund_return_ytd IS NOT NULL
    GROUP BY 1, 2
    ORDER BY 1
)
SELECT * FROM cate_size;
''';

df_sql = pd.read_sql_query(sql,con=engine)
df_sql

,investment_type,size_type,average_return_ytd
0,Blend,Large,0.12
1,Blend,Medium,0.14
2,Blend,Small,0.21
3,Growth,Large,0.11
4,Growth,Medium,0.09
5,Growth,Small,0.07
6,Value,Large,0.15
7,Value,Medium,0.22
8,Value,Small,0.22


The analysis reveals that all fund returns exhibit positive results. Notably, value investment, characterized by its focus on undervalued stocks based on fundamental criteria, consistently outperforms growth investment, which prioritizes capital growth over high income yields. This trend persists across all fund sizes, with value investment demonstrating superior performance regardless of scale.

### 2. Fund Category - Identifying the Top 10 Fund Categories with Highest Alpha Over the Past 3 Years

As SQL lacks a built-in function to directly calculate the median of a column, for the sake of project simplicity, the mean is utilized as an approximation. Verification against the median reveals negligible deviation, thus establishing it as a reasonable alternative.

In [36]:
# query for top 10 alpha by fund category

sql='''
WITH alpha AS (
    SELECT
        fund_category,
        ROUND(AVG(fund_alpha_3years), 2) alpha_3yr
    FROM us_etfs_staging
    WHERE fund_category IS NOT NULL
    GROUP BY 1
    ORDER BY 2 DESC
)
SELECT * FROM alpha
LIMIT 10;
''';

df_sql = pd.read_sql_query(sql,con=engine)
df_sql

,fund_category,alpha_3yr
0,Foreign Small/Mid Growth,9.79
1,Technology,8.31
2,Convertibles,6.96
3,Miscellaneous Sector,6.82
4,China Region,5.65
5,Latin America Stock,5.28
6,Equity Precious Metals,4.47
7,Trading--Inverse Debt,3.80
8,Preferred Stock,3.34
9,Foreign Large Growth,3.25


The alpha_3yr metric gauges a funds performance against benchmarks like the S&P 500 over three years. Funds with high alpha values demonstrate the potential to deliver superior returns to investors, making them attractive options for those seeking to maximize investment returns while minimizing risk exposure.

### 3. Fund Category - Analyzing the Correlation Between Beta and Sharpe Ratio for Various Fund Categories in the Last 3 Years

As SQL lacks a built-in function to directly calculate the median of a column, for the sake of project simplicity, the mean is utilized as an approximation. Verification against the median reveals negligible deviation, thus establishing it as a reasonable alternative.

In [44]:
# query for beta and sharpe by fund category

sql='''
WITH beta_sharpe AS (
    SELECT
        fund_category,
        ROUND(AVG(fund_beta_3years), 2) beta_3yr,
        ROUND(AVG(fund_sharpe_ratio_3years), 2) sharpe_3yr
    FROM us_etfs_staging
    WHERE fund_category IS NOT NULL
    GROUP BY 1
    ORDER BY 3 DESC
)
SELECT * FROM beta_sharpe
LIMIT 20;
''';

df_sql = pd.read_sql_query(sql,con=engine)
df_sql

,fund_category,beta_3yr,sharpe_3yr
0,Inflation-Protected Bond,0.78,1.36
1,Short Government,0.24,1.19
2,Convertibles,1.45,1.16
3,Intermediate Government,0.61,1.13
4,Intermediate-Term Bond,1.01,1.13
5,Short-Term Bond,0.42,1.12
6,Technology,1.17,1.00
7,Large Growth,1.10,1.00
8,Muni National Interm,1.03,0.99
9,Foreign Small/Mid Growth,0.82,0.92


The analysis of beta_3yr and sharpe_3yr metrics reveals an intriguing finding: there exists no positive correlation between a funds volatility, as indicated by its beta value, and its risk-adjusted performance, as measured by the Sharpe ratio, over a three-year period. This implies that higher volatility does not necessarily translate to higher risk-adjusted returns. 

Moreover, within similar levels of volatility or beta, certain ETFs outperform others in terms of Sharpe ratio, indicating that factors beyond volatility alone significantly influence a funds risk-adjusted performance.

## 4. Tableau Visualization <a class="anchor" id="tableau"></a>

Link to Tableau: https://public.tableau.com/app/profile/jonathan.ho6310/viz/USETFsPerformance/Performance